# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch


In [2]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

2022-04-13 23:20:25.467140: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-13 23:20:25.467319: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [6]:
# loading the dataset

train_df = pd.read_csv('q4_data/train.csv')
test_df = pd.read_csv('q4_data/test.csv')
test_label=pd.read_csv('q4_data/test_labels.csv')

parameter=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for para in parameter:
  test_df[para]=test_label[para]

test_df=test_df[test_df.toxic!=-1]

#dataset understanding
train_df.sample(5)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
60674,a26ae6ce84bfd9de,hello i back i read all comments and additions...,0,0,0,0,0,0
39405,6929117575f19219,Stavros Damianides\nHi. I have recently come u...,0,0,0,0,0,0
36349,611d3bc11ad6dc84,"""==Regarding edits made during October 22 2006...",0,0,0,0,0,0
47574,7f1aabaf2d8bc0e3,YOU DELETED KORY SHERMAN NOW YOU HAVE AIDS!,1,0,0,0,0,0
109858,4b8a55e04cf25dca,Deletion review\nI have posted a question at W...,0,0,0,0,0,0


In [7]:
# let's see the total rows in train, test data and the numbers for the various categories
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
#print(train_df[cols_target].sum())


Total rows in test is 63978
Total rows in train is 159571


In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [9]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [10]:
#train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (63978,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [11]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

KeyboardInterrupt: 

In [ ]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [12]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


Downloading: 100%|██████████| 625/625 [00:00<00:00, 113kB/s]
Downloading: 100%|██████████| 226k/226k [00:07<00:00, 33.0kB/s] 
Downloading: 100%|██████████| 455k/455k [00:13<00:00, 34.6kB/s] 
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 468B/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 28.3kB/s]
Downloading:   3%|▎         | 13.4M/418M [00:20<06:26, 1.10MB/s]  

KeyboardInterrupt: 

Downloading:   3%|▎         | 13.5M/418M [00:36<06:26, 1.10MB/s]

In [ ]:
#Obtain X_test and X_train 
x_train_set=X[:100]
x_test_set=test_X[:100]
y_train_set=train_df[:100]
y_test_set=test_df[:100]

x_train=[]
for sentence in x_train_set:
    a=list(get_bert_embedding(sentence))
    x_train.append(a)

x_test=[]
for sentence in x_test_set:
    a=list(get_bert_embedding(sentence))
    x_test.append(a)

In [ ]:
y_train=y_train_set.drop(labels=['id','comment_text'],axis=1)
y_test=y_test_set.drop(labels=['id','comment_text'],axis=1)

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [ ]:
#calculate the accuracies obtained, u can use sklearn metrics


In [ ]:
# Naive Bayes classifier 

In [ ]:
#Support Vector Machines

Which classifier of the three performed better, and why do you think so ?

Ans:

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [ ]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed

def initialise_weights():
  return
  

def loss_function():
  return 


def forward_layer():
  return 


def backward_propogation():
  return 




In [ ]:
def train():
  

In [ ]:
#report accuracies
#here, you need to predict for each of the label as we have trained classifiers in such a way

def predict():




In [ ]:
#report accuracies on the test dataset 





Now, compare your implemented MLP with the sklearn MLP layer.

In [ ]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier



Which MLP performed better? 
Understand and Analyse.